In [28]:
from transformers import AutoProcessor, Blip2ForConditionalGeneration
import torch
from glob import glob
import json
import os
from tqdm import tqdm

In [2]:
processor = AutoProcessor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained(
    "Salesforce/blip2-opt-2.7b", torch_dtype=torch.float16
).to("cuda")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
model.requires_grad_

<bound method Module.requires_grad_ of Blip2ForConditionalGeneration(
  (vision_model): Blip2VisionModel(
    (embeddings): Blip2VisionEmbeddings(
      (patch_embedding): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
    )
    (encoder): Blip2Encoder(
      (layers): ModuleList(
        (0-38): 39 x Blip2EncoderLayer(
          (self_attn): Blip2Attention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=1408, out_features=4224, bias=True)
            (projection): Linear(in_features=1408, out_features=1408, bias=True)
          )
          (layer_norm1): LayerNorm((1408,), eps=1e-05, elementwise_affine=True)
          (mlp): Blip2MLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1408, out_features=6144, bias=True)
            (fc2): Linear(in_features=6144, out_features=1408, bias=True)
          )
          (layer_norm2): LayerNorm((1408,), eps=1e-05, elementwise_affine=True)
        )
      )


In [19]:
path = "/home/aihao/workspace/DeepLearningContent/datasets/images/data/pixiv/**"

In [21]:
file_paths = glob(path, recursive=True)
file_paths

['/home/aihao/workspace/DeepLearningContent/datasets/images/data/pixiv/',
 '/home/aihao/workspace/DeepLearningContent/datasets/images/data/pixiv/我美蘭@土曜日 西あ31a',
 '/home/aihao/workspace/DeepLearningContent/datasets/images/data/pixiv/我美蘭@土曜日 西あ31a/Manga',
 '/home/aihao/workspace/DeepLearningContent/datasets/images/data/pixiv/我美蘭@土曜日 西あ31a/Manga/_我美蘭@土曜日 西あ31a - pixiv__Manga_栄光のガーディアンバトル,work illustration_45581208_p001.jpg',
 '/home/aihao/workspace/DeepLearningContent/datasets/images/data/pixiv/我美蘭@土曜日 西あ31a/Manga/_我美蘭@土曜日 西あ31a - pixiv__Manga_柳生十兵衛茜,鬼武者Soulコンテスト,鬼属性_27103411_p000.jpg',
 "/home/aihao/workspace/DeepLearningContent/datasets/images/data/pixiv/我美蘭@土曜日 西あ31a/Manga/_我美蘭@土曜日 西あ31a - pixiv__Manga_照れ顔いろいろイメレス9!,kino,強力なカウンター,Kino's Journey,怪しいやつが一体いる,incredibly cute,キノの旅100users入り_23096464_p001.jpg",
 '/home/aihao/workspace/DeepLearningContent/datasets/images/data/pixiv/我美蘭@土曜日 西あ31a/Manga/_我美蘭@土曜日 西あ31a - pixiv__Manga_柳生十兵衛茜,鬼武者Soulコンテスト,鬼属性_27103411_p001.jpg',
 '/home/aihao/work

In [26]:
save_path = os.path.join(
    "/home/aihao/workspace/DeepLearningContent/datasets/images/data/pixiv",
    "prompts.json",
)

In [27]:
result = {}

In [29]:

for file_path in tqdm(file_paths):
    try:
        image = Image.open(file_path)
    except IOError:
        continue
    else:
        inputs = processor(image, return_tensors="pt").to("cuda", torch.float16)
        generated_ids = model.generate(**inputs, max_new_tokens=20)
        generated_text = processor.batch_decode(
            generated_ids, skip_special_tokens=True
        )[0].strip()
        result[file_path] = generated_text

100%|██████████| 9257/9257 [32:01<00:00,  4.82it/s] 


In [30]:
result

{'/home/aihao/workspace/DeepLearningContent/datasets/images/data/pixiv/我美蘭@土曜日 西あ31a/Manga/_我美蘭@土曜日 西あ31a - pixiv__Manga_栄光のガーディアンバトル,work illustration_45581208_p001.jpg': 'an anime character with long hair and a dress',
 '/home/aihao/workspace/DeepLearningContent/datasets/images/data/pixiv/我美蘭@土曜日 西あ31a/Manga/_我美蘭@土曜日 西あ31a - pixiv__Manga_柳生十兵衛茜,鬼武者Soulコンテスト,鬼属性_27103411_p000.jpg': 'anime girl with long hair and sword in hand',
 "/home/aihao/workspace/DeepLearningContent/datasets/images/data/pixiv/我美蘭@土曜日 西あ31a/Manga/_我美蘭@土曜日 西あ31a - pixiv__Manga_照れ顔いろいろイメレス9!,kino,強力なカウンター,Kino's Journey,怪しいやつが一体いる,incredibly cute,キノの旅100users入り_23096464_p001.jpg": 'anime characters with different expressions',
 '/home/aihao/workspace/DeepLearningContent/datasets/images/data/pixiv/我美蘭@土曜日 西あ31a/Manga/_我美蘭@土曜日 西あ31a - pixiv__Manga_柳生十兵衛茜,鬼武者Soulコンテスト,鬼属性_27103411_p001.jpg': 'a woman in red and black with a sword',
 '/home/aihao/workspace/DeepLearningContent/datasets/images/data/pixiv/我美蘭@土曜日 西あ31a/Man

In [31]:
with open(save_path, "w") as f:
    json.dump(result, f)

In [33]:
for k,v in result.items():
    print(k)
    print(v)
    break

/home/aihao/workspace/DeepLearningContent/datasets/images/data/pixiv/我美蘭@土曜日 西あ31a/Manga/_我美蘭@土曜日 西あ31a - pixiv__Manga_栄光のガーディアンバトル,work illustration_45581208_p001.jpg
an anime character with long hair and a dress


In [34]:
result2={}

In [35]:
with open(save_path, "r") as f:
    result2=json.load(f)

In [36]:
result2

{'/home/aihao/workspace/DeepLearningContent/datasets/images/data/pixiv/我美蘭@土曜日 西あ31a/Manga/_我美蘭@土曜日 西あ31a - pixiv__Manga_栄光のガーディアンバトル,work illustration_45581208_p001.jpg': 'an anime character with long hair and a dress',
 '/home/aihao/workspace/DeepLearningContent/datasets/images/data/pixiv/我美蘭@土曜日 西あ31a/Manga/_我美蘭@土曜日 西あ31a - pixiv__Manga_柳生十兵衛茜,鬼武者Soulコンテスト,鬼属性_27103411_p000.jpg': 'anime girl with long hair and sword in hand',
 "/home/aihao/workspace/DeepLearningContent/datasets/images/data/pixiv/我美蘭@土曜日 西あ31a/Manga/_我美蘭@土曜日 西あ31a - pixiv__Manga_照れ顔いろいろイメレス9!,kino,強力なカウンター,Kino's Journey,怪しいやつが一体いる,incredibly cute,キノの旅100users入り_23096464_p001.jpg": 'anime characters with different expressions',
 '/home/aihao/workspace/DeepLearningContent/datasets/images/data/pixiv/我美蘭@土曜日 西あ31a/Manga/_我美蘭@土曜日 西あ31a - pixiv__Manga_柳生十兵衛茜,鬼武者Soulコンテスト,鬼属性_27103411_p001.jpg': 'a woman in red and black with a sword',
 '/home/aihao/workspace/DeepLearningContent/datasets/images/data/pixiv/我美蘭@土曜日 西あ31a/Man